In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, Activation, LSTM
from keras.utils import np_utils

import os

# brew install graphviz
# pip3 install graphviz
# pip3 install pydot-ng
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# sample sentence
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence))  # id -> char ['i', 'l', 'e', 'o', 'h', ...]
char_dic = {w: i for i, w in enumerate(char_set)}
print(char_dic)

{'s': 0, '.': 1, 'u': 2, 'd': 3, 'a': 4, 'r': 5, 'g': 6, 'l': 7, 'y': 8, 't': 9, 'k': 10, 'm': 11, 'h': 12, ' ': 13, 'c': 14, 'o': 15, 'n': 16, ',': 17, 'i': 18, 'p': 19, 'b': 20, 'f': 21, 'e': 22, 'w': 23, "'": 24}


In [4]:
data_dim = len(char_set)
seq_length = timesteps = 10
num_classes = len(char_set)

In [5]:
dataX = []
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # char to index
    y = [char_dic[c] for c in y_str]  # char to index

    dataX.append(x)
    dataY.append(y)

if you wan -> f you want
f you want ->  you want 
 you want  -> you want t
you want t -> ou want to
ou want to -> u want to 
u want to  ->  want to b
 want to b -> want to bu
want to bu -> ant to bui
ant to bui -> nt to buil
nt to buil -> t to build
t to build ->  to build 
 to build  -> to build a
to build a -> o build a 
o build a  ->  build a s
 build a s -> build a sh
build a sh -> uild a shi
uild a shi -> ild a ship
ild a ship -> ld a ship,
ld a ship, -> d a ship, 
d a ship,  ->  a ship, d
 a ship, d -> a ship, do
a ship, do ->  ship, don
 ship, don -> ship, don'
ship, don' -> hip, don't
hip, don't -> ip, don't 
ip, don't  -> p, don't d
p, don't d -> , don't dr
, don't dr ->  don't dru
 don't dru -> don't drum
don't drum -> on't drum 
on't drum  -> n't drum u
n't drum u -> 't drum up
't drum up -> t drum up 
t drum up  ->  drum up p
 drum up p -> drum up pe
drum up pe -> rum up peo
rum up peo -> um up peop
um up peop -> m up peopl
m up peopl ->  up people
 up people -> up people 


In [6]:
# One-hot encoding
dataX = np_utils.to_categorical(dataX, num_classes=num_classes)
# reshape X to be [samples, time steps, features]
dataX = np.reshape(dataX, (-1, seq_length, data_dim))
print(dataX.shape)

# One-hot encoding
dataY = np_utils.to_categorical(dataY, num_classes=num_classes)
# time steps
dataY = np.reshape(dataY, (-1, seq_length, data_dim))
print(dataY.shape)

(170, 10, 25)
(170, 10, 25)


In [8]:
model = Sequential()
model.add(LSTM(num_classes, input_shape=(timesteps, data_dim), return_sequences=True))
model.add(LSTM(num_classes, return_sequences=True))
model.add(TimeDistributed(Dense(num_classes)))

model.add(Activation('softmax'))
model.summary()

# Store model graph in png
# (Error occurs on in python interactive shell)
#plot_model(model, to_file=os.path.basename(__file__) + '.png', show_shapes=True)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', metrics=['accuracy'])

W0315 12:43:22.236589  9244 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0315 12:43:22.326920  9244 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 25)            650       
_________________________________________________________________
activation_2 (Activation)    (None, 10, 25)            0         
Total params: 10,850
Trainable params: 10,850
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(dataX, dataY, epochs=1000)

W0315 12:43:26.115080  9244 deprecation.py:323] From C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0315 12:43:32.208697  9244 deprecation_wrapper.py:119] From C:\Users\Admin\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1000
170/170 [==============================] - 5s 30ms/step - loss: 3.2062 - acc: 0.1141
Epoch 2/1000
170/170 [==============================] - 0s 1ms/step - loss: 3.1737 - acc: 0.1588
Epoch 3/1000
170/170 [==============================] - 0s 1ms/step - loss: 3.1222 - acc: 0.1447
Epoch 4/1000
170/170 [==============================] - 0s 1ms/step - loss: 3.0524 - acc: 0.1824
Epoch 5/1000
170/170 [==============================] - 0s 1ms/step - loss: 3.0046 - acc: 0.1894
Epoch 6/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.9798 - acc: 0.1900
Epoch 7/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.9618 - acc: 0.1894
Epoch 8/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.9476 - acc: 0.1894
Epoch 9/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.9351 - acc: 0.1894
Epoch 10/1000
170/170 [==============================] - 0s 974us/step - loss: 2.9244 - acc: 0.1894
Epoch 11/1000
170/170 [===

170/170 [==============================] - 0s 1ms/step - loss: 2.2561 - acc: 0.3371
Epoch 85/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.2407 - acc: 0.3441
Epoch 86/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.2262 - acc: 0.3476
Epoch 87/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.2168 - acc: 0.3476
Epoch 88/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.2039 - acc: 0.3524A: 0s - loss: 2.2006 - acc: 0.354
Epoch 89/1000
170/170 [==============================] - ETA: 0s - loss: 2.1938 - acc: 0.357 - 0s 1ms/step - loss: 2.1932 - acc: 0.3612
Epoch 90/1000
170/170 [==============================] - 0s 906us/step - loss: 2.1797 - acc: 0.3612
Epoch 91/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.1668 - acc: 0.3612
Epoch 92/1000
170/170 [==============================] - 0s 1ms/step - loss: 2.1557 - acc: 0.3676
Epoch 93/1000
170/170 [==============================] - 0s

170/170 [==============================] - 0s 1ms/step - loss: 1.2930 - acc: 0.6518
Epoch 167/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2872 - acc: 0.6488
Epoch 168/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2733 - acc: 0.6559
Epoch 169/1000
170/170 [==============================] - 0s 2ms/step - loss: 1.2627 - acc: 0.6618
Epoch 170/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2534 - acc: 0.6647
Epoch 171/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2463 - acc: 0.6629
Epoch 172/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2320 - acc: 0.6735
Epoch 173/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2253 - acc: 0.6688A: 0s - loss: 1.2409 - acc: 0.66
Epoch 174/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2145 - acc: 0.6829
Epoch 175/1000
170/170 [==============================] - 0s 1ms/step - loss: 1.2041 - acc: 

170/170 [==============================] - 0s 1ms/step - loss: 0.6663 - acc: 0.8276
Epoch 248/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6605 - acc: 0.8282
Epoch 249/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6576 - acc: 0.8276
Epoch 250/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6528 - acc: 0.8318
Epoch 251/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6468 - acc: 0.8312
Epoch 252/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6418 - acc: 0.8353
Epoch 253/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6385 - acc: 0.8341
Epoch 254/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6362 - acc: 0.8341
Epoch 255/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6304 - acc: 0.8382
Epoch 256/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.6272 - acc: 0.8365
Epoch 257/1000
170/170 [=

...

170/170 [==============================] - 0s 1ms/step - loss: 0.2590 - acc: 0.8924
Epoch 656/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2581 - acc: 0.8924
Epoch 657/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2579 - acc: 0.8876
Epoch 658/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2588 - acc: 0.8935
Epoch 659/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2586 - acc: 0.8900
Epoch 660/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2580 - acc: 0.8941
Epoch 661/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2580 - acc: 0.8871
Epoch 662/1000
170/170 [==============================] - 0s 863us/step - loss: 0.2578 - acc: 0.8929
Epoch 663/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2580 - acc: 0.8941
Epoch 664/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2574 - acc: 0.8953
Epoch 665/1000
170/170 

170/170 [==============================] - 0s 1ms/step - loss: 0.2526 - acc: 0.8906
Epoch 738/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2515 - acc: 0.8871
Epoch 739/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2520 - acc: 0.8894
Epoch 740/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2520 - acc: 0.8906
Epoch 741/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2519 - acc: 0.8888
Epoch 742/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2523 - acc: 0.8865
Epoch 743/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2525 - acc: 0.8894
Epoch 744/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2517 - acc: 0.8888
Epoch 745/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2520 - acc: 0.8853
Epoch 746/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2521 - acc: 0.8894
Epoch 747/1000
170/170 [=

170/170 [==============================] - 0s 1ms/step - loss: 0.2477 - acc: 0.8841
Epoch 820/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2484 - acc: 0.8865
Epoch 821/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2485 - acc: 0.8906
Epoch 822/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2485 - acc: 0.8918
Epoch 823/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2482 - acc: 0.8882
Epoch 824/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2474 - acc: 0.8918
Epoch 825/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2480 - acc: 0.8882A: 0s - loss: 0.2473 - acc: 0.888
Epoch 826/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2477 - acc: 0.8924
Epoch 827/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2474 - acc: 0.8906
Epoch 828/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2474 - acc:

170/170 [==============================] - 0s 1ms/step - loss: 0.2451 - acc: 0.8918
Epoch 901/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2458 - acc: 0.8894
Epoch 902/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2457 - acc: 0.8876
Epoch 903/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2453 - acc: 0.8894
Epoch 904/1000
170/170 [==============================] - ETA: 0s - loss: 0.2425 - acc: 0.888 - 0s 1ms/step - loss: 0.2453 - acc: 0.8882
Epoch 905/1000
170/170 [==============================] - 0s 876us/step - loss: 0.2453 - acc: 0.8888
Epoch 906/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2449 - acc: 0.8847
Epoch 907/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2454 - acc: 0.8888
Epoch 908/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2456 - acc: 0.8888
Epoch 909/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2453

170/170 [==============================] - 0s 1ms/step - loss: 0.2438 - acc: 0.8906
Epoch 982/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2440 - acc: 0.8882
Epoch 983/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2432 - acc: 0.8824
Epoch 984/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2438 - acc: 0.8871
Epoch 985/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2436 - acc: 0.8906
Epoch 986/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2427 - acc: 0.8882
Epoch 987/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2432 - acc: 0.8912
Epoch 988/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2431 - acc: 0.8912
Epoch 989/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2435 - acc: 0.8912
Epoch 990/1000
170/170 [==============================] - 0s 1ms/step - loss: 0.2429 - acc: 0.8900
Epoch 991/1000
170/170 [=

In [10]:
predictions = model.predict(dataX, verbose=0)
for i, prediction in enumerate(predictions):
    # print(prediction)
    x_index = np.argmax(dataX[i], axis=1)
    x_str = [char_set[j] for j in x_index]

    index = np.argmax(prediction, axis=1)
    result = [char_set[j] for j in index]

    print(''.join(x_str), ' -> ', ''.join(result))

if you wan  ->  g you want
f you want  ->  oyou want 
 you want   ->  tou want t
you want t  ->  ou want to
ou want to  ->    want to 
u want to   ->  iwant to b
 want to b  ->  tont to bu
want to bu  ->  ont to bui
ant to bui  ->  nd to buil
nt to buil  ->  d to build
t to build  ->  hto build 
 to build   ->  to cuild a
to build a  ->  h luild a 
o build a   ->   luild a s
 build a s  ->  tutld a sh
build a sh  ->  utld a shi
uild a shi  ->  ild a ship
ild a ship  ->  gd a ship,
ld a ship,  ->  e a ship, 
d a ship,   ->   a ship, d
 a ship, d  ->  tnship, do
a ship, do  ->  nship, don
 ship, don  ->  thip, don'
ship, don'  ->   ip, don't
hip, don't  ->  ep, don't 
ip, don't   ->  g, don't d
p, don't d  ->  e don't dr
, don't dr  ->   don't dru
 don't dru  ->  ton't drum
don't drum  ->   n't drum 
on't drum   ->   't drum u
n't drum u  ->  dt drum up
't drum up  ->  t drum up 
t drum up   ->  htrum up p
 drum up p  ->  toum up pe
drum up pe  ->   um up peo
rum up peo  ->   m up peop
u